In [43]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from IPython.display import display


In [44]:
data = np.load("./data/generated_languages.npz", allow_pickle=True)
generated_languages = np.squeeze(data["generated_languages"])

generated_cons_pl = generated_languages[:, 0:14, 0:24]
generated_cons_npl = generated_languages[:, 15:22, 0:10]
generated_cons_coart = generated_languages[:, 15:18, 10:14]
generated_vowels = generated_languages[:, 15:22, 14:24]

generated_languages.shape, generated_cons_pl.shape, generated_cons_npl.shape, generated_cons_coart.shape, generated_vowels.shape

((18, 24, 24), (18, 14, 24), (18, 7, 10), (18, 3, 4), (18, 7, 10))

In [45]:
# load the phoneme table dataframes

cons_pl_tbl_df = pd.read_csv(
    "./data/consonants_plumonic.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
    on_bad_lines="warn",
)

cons_npl_tbl_df = pd.read_csv(
    "./data/consonants_non_plumonic.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)

cons_coart_tbl_df = pd.read_csv(
    "./data/consonants_coarticulated.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)

vowels_tbl_df = pd.read_csv(
    "./data/vowels.csv",
    dtype=str,
    index_col=[0],
    keep_default_na=False,
    na_values=["-1"],
)

cons_pl_tbl_df.shape, cons_npl_tbl_df.shape, cons_coart_tbl_df.shape, vowels_tbl_df.shape

((14, 24), (7, 10), (3, 4), (7, 10))

In [52]:
l = generated_languages.shape[0]

def map_generated_to_phonemes(tbl):
    def mapper(lang):
        return np.where(lang > 0, tbl.replace(np.nan, "*").replace("", "*"), "")
    return mapper
    
generated_cons_pl_phonemes = np.array([map_generated_to_phonemes(cons_pl_tbl_df)(p) for p in generated_cons_pl])
generated_cons_npl_phonemes = np.array([map_generated_to_phonemes(cons_npl_tbl_df)(p) for p in generated_cons_npl])
generated_cons_coart_phonemes = np.array([map_generated_to_phonemes(cons_coart_tbl_df)(p) for p in generated_cons_coart])
generated_vowels_phonemes = np.array([map_generated_to_phonemes(vowels_tbl_df)(p) for p in generated_vowels])

display(pd.DataFrame(generated_cons_pl_phonemes[5], index=cons_pl_tbl_df.index, columns=cons_pl_tbl_df.columns))
display(pd.DataFrame(generated_cons_npl_phonemes[5], index=cons_npl_tbl_df.index, columns=cons_npl_tbl_df.columns))
display(pd.DataFrame(generated_cons_coart_phonemes[5], index=cons_coart_tbl_df.index, columns=cons_coart_tbl_df.columns))
display(pd.DataFrame(generated_vowels_phonemes[5], index=vowels_tbl_df.index, columns=vowels_tbl_df.columns))


generated_cons_pl_phonemes.shape, generated_cons_npl_phonemes.shape, generated_cons_coart_phonemes.shape, generated_vowels_phonemes.shape

,Bilabial,Bilabial_v,Labiodental,Labiodental_v,Linguolabial,Linguolabial_v,Dental,Dental_v,Alveolar,Alveolar_v,...,Palatal,Palatal_v,Velar,Velar_v,Uvular,Uvular_v,Pharyngeal,Pharyngeal_v,Glottal,Glottal_v
Nasal,,m,,,,,,,,n,...,,,,ŋ,,,,,,
Plosive,p,,,,,,,,,,...,,,k,ɡ,,,,,,
Sibilant affricate,,,,,,,,,,,...,,,,,,,,,,
Non-sibilant affricate,,,,,,,,,,,...,,,,,,,,,,
Sibilant fricative,,,,,,,,,,,...,,,,,,,,,,
Non-sibilant fricative,,β,f,,,,,,,,...,,,x,ɣ,,,,,h,
Approximant,,,,,,,,,,ɹ,...,,j,,,,,,,,
Tap/flap,,,,,,,,,,,...,,,,,,,,,,
Trill,,,,,,,,,,,...,,,,,,,,,,
Fricative,,,f,,,,,,,,...,,,,,,,,,h,


,Bilabial,Labiodental,Dental,Alveolar,Palato-alveolar,Retroflex,Palatal,Velar,Uvular,Pharyngeal
Ejective Stop,,,,,,,,,,
Ejective Fricative,,,,,,,,,,
Ejective Affricate,,,,,,,,,,
Ejective Lateral,,,,,,,,,,
Click,,,,,,,,,,
Implosive Voiced,,,,,,,,,,
Implosive Voiceless,,,,,,,,,,


,Labial–palatal,Labial–velar,Velarized alveolar,Palatal-velar
Fricative,,,,
Approximant,,w,,
Lateral approximant,,,,


,Front unrounded,Front rounded,Near-front unrounded,Near-front rounded,Central unrounded,Central rounded,Near-back unrounded,Near-back rounded,Back unrounded,Back rounded
Close,i,,,,,,,,,u
Near-close,,,,,,,,,,
Close-mid,e,,,,,,,,,o
Mid,,,,,,,,,,
Open-mid,,,,,,,,,,ɔ
Near-open,,,,,,,,,,
Open,a,,,,,,,,,


((18, 14, 24), (18, 7, 10), (18, 3, 4), (18, 7, 10))